# 1. 自动识别

In [1]:
import os

In [2]:
list_file = list(os.walk('./input'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
for file in list_file:
    if '整理好的替代料总表' in file:
        alternate = './input/' + file
    elif '改价表' in file:
        price = './input/' + file
    elif '替代料总表' in file and '整理好的' not in file:
        combination = './input/' + file
    elif 'MRP' in file:
        mrp = './input/' + file
    elif '划分' in file:
        group = './input/' + file
    elif '断货改价' in file:
        last = './input/' + file

print('alternate:     ' + alternate)
print('price:         ' + price)
print('combination:   ' + combination)
print('mrp:           ' + mrp)
print('group:         ' + group)
print('last:          ' + last)

alternate:     ./input/整理好的替代料总表_20240423_095540.xlsx
price:         ./input/改价表 4-30.xlsx
combination:   ./input/替代料总表更新版4.16.xlsx
mrp:           ./input/US MRP20240422-运营.xlsx
group:         ./input/US-eBay 品类划分 2024.4.16.xlsx
last:          ./input/2U-断货改价-us-下午 4.29.xlsx


# 2. 在库在途计算
## 2.1. 替代料计算

In [3]:
import pandas as pd

In [4]:
df_alternate = pd.read_excel(alternate,
                             header=0,
                             dtype=str).fillna('')

df_alternate['row'] = [i+1 for i in range(len(df_alternate))]

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,row
0,USAC64980T,USAC64980,,,,,1
1,USACC02805S-C,USACC02805,USACC02805S-C1,,,,2
2,USACC29325-C,USACC29325,USACC29325-C1,,,,3
3,USACH11594J-C,USACH11594,,,,,4
4,USOC51647,USOC51647-C,,,,,5
...,...,...,...,...,...,...,...
9411,USOC39409,USOC39409-C,,,,,9412
9412,USOC3C037,USOC3C037-C,,,,,9413
9413,USOC42090,USOC42090-C,,,,,9414
9414,USOC47079,USOC47079-C,,,,,9415


In [5]:
# 替代料表变形
list_df = []
for column in df_alternate.columns.tolist()[:-1]:
    df_temp =  df_alternate[['主料SKU', column, 'row']]
    df_temp.columns = ['主料', 'SKU', 'row']
    df_temp = df_temp[df_temp['SKU'] != '']
    list_df.append(df_temp)

df_alternate = pd.concat(list_df, ignore_index=True)

df_alternate.to_excel('./output/1.position.xlsx', index=False)

df_alternate

,主料,SKU,row
0,USAC64980T,USAC64980T,1
1,USACC02805S-C,USACC02805S-C,2
2,USACC29325-C,USACC29325-C,3
3,USACH11594J-C,USACH11594J-C,4
4,USOC51647,USOC51647,5
...,...,...,...
21122,USGS62390GUB,USGS62390GUB-C,8239
21123,USGS6310TNZB-C,USGS6310NZ,8288
21124,USGS64520TNZ-C,USGS64520,8370
21125,USGS65180TB-C,USGS65180T,8407


In [6]:
df_price = pd.read_excel(price,
                         sheet_name=0,
                         header=0,
                         names=['SKU', '在途', '在库'],
                         usecols='A, E, F',
                         dtype=str).fillna({'SKU': '',
                                            '在途': 0,
                                            '在库': 0})

df_price = df_price[df_price['SKU'] != ''].reset_index(drop=True)

df_price['在库'] = df_price['在库'].astype(int)
df_price['在途'] = df_price['在途'].astype(int)

# 处理后缀
# -N：原单非质量、无理由 等问题的售后退件
# -NC：原单投诉质量问题未核实、货值比较大的订单；原单非质量问题但产品略有小瑕疵（一丢丢脏）
# -D：原单不是质量问题，但退件有缺失一些小部件
df_price['SKU'] = [sku.strip()[:-3] if sku.strip().endswith('-NC') else sku.strip()[:-2] if sku.strip().endswith('-N') else sku.strip()[:-2] if sku.strip().endswith('-D') else sku.strip() for sku in df_price['SKU'].tolist()]

df_price = df_price.groupby([df_price['SKU']], as_index=False).sum()

df_price

,SKU,在途,在库
0,BCB96454-XH,0,0
1,BPD14040-XH,0,0
2,BPD8550086500+BRS31318,0,0
3,BRR34279-XH,0,0
4,CABM0021A,0,0
...,...,...,...
111853,YHTWS58202-Y-6,0,0
111854,YHTWS5W1ZBA-Y-10,0,0
111855,YHTWS5W1ZBA-Y-8,0,0
111856,YHTWS802AA-Y-6,0,0


In [7]:
print(df_alternate.columns)
print(df_price.columns)

# 找出有替代料关系的数据
df_temp = pd.merge(left=df_alternate,
                   right=df_price,
                   how='left',
                   left_on='SKU',
                   right_on='SKU').fillna(0)


df_temp

Index(['主料', 'SKU', 'row'], dtype='object')
Index(['SKU', '在途', '在库'], dtype='object')


,主料,SKU,row,在途,在库
0,USAC64980T,USAC64980T,1,0.0,3.0
1,USACC02805S-C,USACC02805S-C,2,0.0,0.0
2,USACC29325-C,USACC29325-C,3,0.0,0.0
3,USACH11594J-C,USACH11594J-C,4,8.0,2.0
4,USOC51647,USOC51647,5,0.0,0.0
...,...,...,...,...,...
21122,USGS62390GUB,USGS62390GUB-C,8239,0.0,0.0
21123,USGS6310TNZB-C,USGS6310NZ,8288,0.0,0.0
21124,USGS64520TNZ-C,USGS64520,8370,0.0,0.0
21125,USGS65180TB-C,USGS65180T,8407,0.0,0.0


In [8]:
# 合并有替代料关系的数据
df_temp = df_temp[['row', '在库', '在途']]
df_temp = df_temp.groupby([df_temp['row']], as_index=False).sum()

df_temp

,row,在库,在途
0,1,3.0,0.0
1,2,21.0,42.0
2,3,25.0,34.0
3,4,3.0,10.0
4,5,21.0,0.0
...,...,...,...
9411,9412,5.0,0.0
9412,9413,1.0,6.0
9413,9414,10.0,15.0
9414,9415,33.0,4.0


In [9]:
print(df_alternate.columns)
print(df_temp.columns)

# 更新有替代料关系的数据
df_temp = pd.merge(left=df_alternate,
                   right=df_temp,
                   how='left',
                   left_on='row',
                   right_on='row').fillna(0)

df_temp

Index(['主料', 'SKU', 'row'], dtype='object')
Index(['row', '在库', '在途'], dtype='object')


,主料,SKU,row,在库,在途
0,USAC64980T,USAC64980T,1,3.0,0.0
1,USACC02805S-C,USACC02805S-C,2,21.0,42.0
2,USACC29325-C,USACC29325-C,3,25.0,34.0
3,USACH11594J-C,USACH11594J-C,4,3.0,10.0
4,USOC51647,USOC51647,5,21.0,0.0
...,...,...,...,...,...
21122,USGS62390GUB,USGS62390GUB-C,8239,25.0,0.0
21123,USGS6310TNZB-C,USGS6310NZ,8288,30.0,11.0
21124,USGS64520TNZ-C,USGS64520,8370,32.0,52.0
21125,USGS65180TB-C,USGS65180T,8407,2.0,16.0


In [10]:
print(df_price.columns)
print(df_alternate.columns)

# 找出没有替代料关系的数据
df_temp_2 = pd.merge(left=df_price,
                     right=df_alternate,
                     how='left',
                     left_on='SKU',
                     right_on='SKU').fillna('')

df_temp_2 = df_temp_2[df_temp_2['row'] == '']
df_temp_2['主料'] = df_temp_2['SKU'].tolist()
df_temp_2 = df_temp_2[['主料', 'SKU', 'row', '在库', '在途']]

df_temp_2

Index(['SKU', '在途', '在库'], dtype='object')
Index(['主料', 'SKU', 'row'], dtype='object')


,主料,SKU,row,在库,在途
0,BCB96454-XH,BCB96454-XH,,0,0
1,BPD14040-XH,BPD14040-XH,,0,0
2,BPD8550086500+BRS31318,BPD8550086500+BRS31318,,0,0
3,BRR34279-XH,BRR34279-XH,,0,0
4,CABM0021A,CABM0021A,,0,0
...,...,...,...,...,...
111853,YHTWS58202-Y-6,YHTWS58202-Y-6,,0,0
111854,YHTWS5W1ZBA-Y-10,YHTWS5W1ZBA-Y-10,,0,0
111855,YHTWS5W1ZBA-Y-8,YHTWS5W1ZBA-Y-8,,0,0
111856,YHTWS802AA-Y-6,YHTWS802AA-Y-6,,0,0


In [11]:
# 整合数据
df_price = pd.concat([df_temp, df_temp_2], ignore_index=True)
df_price['new_row'] = [i+1 for i in range(len(df_price), len(df_price) * 2)]
df_price['row'] = [row if row != '' else new_row for row, new_row in zip(df_price['row'].tolist(), df_price['new_row'].tolist())]
df_price = df_price[['主料', 'SKU', 'row', '在库', '在途']]

df_price

,主料,SKU,row,在库,在途
0,USAC64980T,USAC64980T,1,3.0,0.0
1,USACC02805S-C,USACC02805S-C,2,21.0,42.0
2,USACC29325-C,USACC29325-C,3,25.0,34.0
3,USACH11594J-C,USACH11594J-C,4,3.0,10.0
4,USOC51647,USOC51647,5,21.0,0.0
...,...,...,...,...,...
115936,YHTWS58202-Y-6,YHTWS58202-Y-6,231878,0.0,0.0
115937,YHTWS5W1ZBA-Y-10,YHTWS5W1ZBA-Y-10,231879,0.0,0.0
115938,YHTWS5W1ZBA-Y-8,YHTWS5W1ZBA-Y-8,231880,0.0,0.0
115939,YHTWS802AA-Y-6,YHTWS802AA-Y-6,231881,0.0,0.0


## 2.2. 组合计算

In [12]:
df_combination = pd.read_excel(combination,
                               header=1,
                               dtype=str)[['主料SKU', '数量', 'SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5']].fillna('')

df_combination['数量'] = [amount.strip() for amount in df_combination['数量'].tolist()]
df_combination = df_combination[(df_combination['数量'] != '') & (df_combination['数量'] != '1')].reset_index(drop=True)

df_combination

,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5
0,USBDA04233-2-C,2,USBDA04233,,,,
1,USBDA26133-2,2,USBDA26133-C,,,,
2,USBDA26291-2-C,2,USBDA26291,,,,
3,USCAF10002-2-C,2,USCAF10002-C,,,,
4,USCAF10022-2-C,2,USCAF10022-C,,,,
...,...,...,...,...,...,...,...
511,USTCV18195LR-2-C,1+1,USTCV18195-C,USTCV17294,,,
512,USEAF12879-2-C,2,USEAF12879-C,,,,
513,USTCV16803IE-2-C,1+1,USTCV16803-C,USTCV18155-C,,,
514,USTCV18033IE-2-C,1+1,USTCV18033,USTCV18034,,,


In [13]:
print(df_combination.columns)
print(df_price.columns)

df_combination = pd.merge(left=df_combination,
                          right=df_price,
                          how='left',
                          left_on='主料SKU',
                          right_on='SKU').fillna('')

df_combination

Index(['主料SKU', '数量', 'SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5'], dtype='object')
Index(['主料', 'SKU', 'row', '在库', '在途'], dtype='object')


,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5,主料,SKU,row,在库,在途
0,USBDA04233-2-C,2,USBDA04233,,,,,USBDA04233-2-C,USBDA04233-2-C,3165,30.0,0.0
1,USBDA26133-2,2,USBDA26133-C,,,,,USBDA26133-2,USBDA26133-2,3380,3.0,26.0
2,USBDA26291-2-C,2,USBDA26291,,,,,USBDA26291-2-C,USBDA26291-2-C,3381,13.0,0.0
3,USCAF10002-2-C,2,USCAF10002-C,,,,,USCAF10002-2-C,USCAF10002-2-C,160755,8.0,0.0
4,USCAF10022-2-C,2,USCAF10022-C,,,,,USCAF10022-2-C,USCAF10022-2-C,3993,4.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...
511,USTCV18195LR-2-C,1+1,USTCV18195-C,USTCV17294,,,,USTCV18195LR-2-C,USTCV18195LR-2-C,219065,0.0,5.0
512,USEAF12879-2-C,2,USEAF12879-C,,,,,USEAF12879-2-C,USEAF12879-2-C,1227,7.0,8.0
513,USTCV16803IE-2-C,1+1,USTCV16803-C,USTCV18155-C,,,,USTCV16803IE-2-C,USTCV16803IE-2-C,218933,8.0,11.0
514,USTCV18033IE-2-C,1+1,USTCV18033,USTCV18034,,,,USTCV18033IE-2-C,USTCV18033IE-2-C,219017,12.0,3.0


In [14]:
df_combination = pd.merge(left=df_combination,
                   right=df_price[['SKU', '在库', '在途']],
                   how='left',
                   left_on='SKU1',
                   right_on='SKU',
                   suffixes=('', '_1')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_price[['SKU', '在库', '在途']],
                   how='left',
                   left_on='SKU2',
                   right_on='SKU',
                   suffixes=('', '_2')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_price[['SKU', '在库', '在途']],
                   how='left',
                   left_on='SKU3',
                   right_on='SKU',
                   suffixes=('', '_3')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_price[['SKU', '在库', '在途']],
                   how='left',
                   left_on='SKU4',
                   right_on='SKU',
                   suffixes=('', '_4')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_price[['SKU', '在库', '在途']],
                   how='left',
                   left_on='SKU5',
                   right_on='SKU',
                   suffixes=('', '_5')).fillna('')

df_combination

,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5,主料,SKU,row,...,在途_2,SKU_3,在库_3,在途_3,SKU_4,在库_4,在途_4,SKU_5,在库_5,在途_5
0,USBDA04233-2-C,2,USBDA04233,,,,,USBDA04233-2-C,USBDA04233-2-C,3165,...,,,,,,,,,,
1,USBDA26133-2,2,USBDA26133-C,,,,,USBDA26133-2,USBDA26133-2,3380,...,,,,,,,,,,
2,USBDA26291-2-C,2,USBDA26291,,,,,USBDA26291-2-C,USBDA26291-2-C,3381,...,,,,,,,,,,
3,USCAF10002-2-C,2,USCAF10002-C,,,,,USCAF10002-2-C,USCAF10002-2-C,160755,...,,,,,,,,,,
4,USCAF10022-2-C,2,USCAF10022-C,,,,,USCAF10022-2-C,USCAF10022-2-C,3993,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,USTCV18195LR-2-C,1+1,USTCV18195-C,USTCV17294,,,,USTCV18195LR-2-C,USTCV18195LR-2-C,219065,...,27.0,,,,,,,,,
512,USEAF12879-2-C,2,USEAF12879-C,,,,,USEAF12879-2-C,USEAF12879-2-C,1227,...,,,,,,,,,,
513,USTCV16803IE-2-C,1+1,USTCV16803-C,USTCV18155-C,,,,USTCV16803IE-2-C,USTCV16803IE-2-C,218933,...,2.0,,,,,,,,,
514,USTCV18033IE-2-C,1+1,USTCV18033,USTCV18034,,,,USTCV18033IE-2-C,USTCV18033IE-2-C,219017,...,28.0,,,,,,,,,


In [15]:
from tqdm import tqdm
import math

In [16]:
for i in tqdm(range(len(df_combination)), desc='Progress', ncols=77):
    list_amount = [int(amount.strip()) for amount in df_combination.loc[i, '数量'].split('+')]
    list_stock = []
    list_way = []
    for j in range(len(list_amount)):
        list_stock.append(0 if df_combination.loc[i, '在库_'+str(j+1)] == '' else df_combination.loc[i, '在库_'+str(j+1)] / list_amount[j])
        list_way.append(0 if df_combination.loc[i, '在途_'+str(j+1)] == '' else df_combination.loc[i, '在途_'+str(j+1)] / list_amount[j])
    df_combination.loc[i, '组合在库'] = math.floor(min(list_stock))
    df_combination.loc[i, '组合在途'] = math.floor(min(list_way))
        
df_combination = df_combination[['主料SKU', 'row', '组合在库', '组合在途']]
df_combination = df_combination.sort_values(by=['组合在库', '组合在途'], ascending=[False, False]).reset_index(drop=True)

df_combination

Progress: 100%|██████████████████████████| 516/516 [00:00<00:00, 4730.99it/s]


,主料SKU,row,组合在库,组合在途
0,USCAF99554-2-C,4124,87.0,40.0
1,USCAF99594R-2-C,4124,87.0,40.0
2,USEAF11952-2-C,179919,75.0,0.0
3,USBM002A0A-C,1080,75.0,0.0
4,USCAF10285-2-C,160781,65.0,78.0
...,...,...,...,...
511,USGS1038LR,1102,0.0,0.0
512,USGS4020LR,1103,0.0,0.0
513,USGS4170LR,1104,0.0,0.0
514,USTCV00177-2,1220,0.0,0.0


In [17]:
# 找出有替代料关系的数据
df_temp = df_combination[df_combination['row'] != ''].reset_index(drop=True)
df_temp = df_temp.drop_duplicates(['row'], keep='first', ignore_index=True)

print(df_price.columns)
print(df_combination.columns)

df_temp = pd.merge(left=df_price,
                   right=df_temp,
                   how='left',
                   left_on='row',
                   right_on='row').fillna(0)

df_temp = df_temp[['主料', 'SKU', 'row', '在库', '在途', '组合在库', '组合在途']]

df_temp

Index(['主料', 'SKU', 'row', '在库', '在途'], dtype='object')
Index(['主料SKU', 'row', '组合在库', '组合在途'], dtype='object')


,主料,SKU,row,在库,在途,组合在库,组合在途
0,USAC64980T,USAC64980T,1,3.0,0.0,0.0,0.0
1,USACC02805S-C,USACC02805S-C,2,21.0,42.0,0.0,0.0
2,USACC29325-C,USACC29325-C,3,25.0,34.0,0.0,0.0
3,USACH11594J-C,USACH11594J-C,4,3.0,10.0,0.0,0.0
4,USOC51647,USOC51647,5,21.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
115936,YHTWS58202-Y-6,YHTWS58202-Y-6,231878,0.0,0.0,0.0,0.0
115937,YHTWS5W1ZBA-Y-10,YHTWS5W1ZBA-Y-10,231879,0.0,0.0,0.0,0.0
115938,YHTWS5W1ZBA-Y-8,YHTWS5W1ZBA-Y-8,231880,0.0,0.0,0.0,0.0
115939,YHTWS802AA-Y-6,YHTWS802AA-Y-6,231881,0.0,0.0,0.0,0.0


In [18]:
from datetime import datetime

In [19]:
# 找出没有替代料关系的数据
df_temp_2 = df_combination[df_combination['row'] == ''].reset_index(drop=True)
df_temp_2 = df_temp_2.drop_duplicates(['主料SKU'], keep='first', ignore_index=True)
df_temp_2['row'] = [i+1 for i in range(len(df_price) * 2, len(df_price) * 2 + len(df_temp_2))]
df_temp_2['主料'] = df_temp_2['主料SKU'].tolist()
df_temp_2['SKU'] = df_temp_2['主料SKU'].tolist()
df_temp_2['在库'] = 0
df_temp_2['在途'] = 0
df_temp_2 = df_temp_2[['主料', 'SKU', 'row', '在库', '在途', '组合在库', '组合在途']]

if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2[['SKU', '组合在库', '组合在途']].to_excel('./output/warning-1.组合缺失信息-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

,主料,SKU,row,在库,在途,组合在库,组合在途


In [20]:
# 整合数据
df_price = pd.concat([df_temp, df_temp_2], ignore_index=True)

df_price.to_excel('./output/2.stock.xlsx', index=False)

df_price

,主料,SKU,row,在库,在途,组合在库,组合在途
0,USAC64980T,USAC64980T,1.0,3.0,0.0,0.0,0.0
1,USACC02805S-C,USACC02805S-C,2.0,21.0,42.0,0.0,0.0
2,USACC29325-C,USACC29325-C,3.0,25.0,34.0,0.0,0.0
3,USACH11594J-C,USACH11594J-C,4.0,3.0,10.0,0.0,0.0
4,USOC51647,USOC51647,5.0,21.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
115936,YHTWS58202-Y-6,YHTWS58202-Y-6,231878.0,0.0,0.0,0.0,0.0
115937,YHTWS5W1ZBA-Y-10,YHTWS5W1ZBA-Y-10,231879.0,0.0,0.0,0.0,0.0
115938,YHTWS5W1ZBA-Y-8,YHTWS5W1ZBA-Y-8,231880.0,0.0,0.0,0.0,0.0
115939,YHTWS802AA-Y-6,YHTWS802AA-Y-6,231881.0,0.0,0.0,0.0,0.0


# 3. MRP整理

In [21]:
df_mrp = pd.read_excel(mrp,
                       header=1,
                       names=['SKU', '品类', '层级', '身份', '未售出周数', '在途总数', 'W7', 'W6', 'W5', 'W4', 'W3', 'W2', 'W1', 'MRP周销'],
                       usecols='B, F, G, I, K, P, V:AB, AD',
                       dtype=str).fillna('')

df_group = pd.read_excel(group,
                         header=0,
                         dtype=str).fillna('')

df_mrp = pd.merge(left=df_mrp,
                  right=df_group,
                  how='left',
                  left_on='品类',
                  right_on='品类').fillna('')

df_mrp = df_mrp[df_mrp['品类负责人'] == 'Ada'].reset_index(drop=True)

df_mrp

,SKU,品类,层级,身份,未售出周数,在途总数,W7,W6,W5,W4,W3,W2,W1,MRP周销,品类负责人
0,USTCV17215HIE-2-C,TCV,TOP1,老品,0,195,63,38,41,45,36,38,60,45.5,Ada
1,USOBDAD100-C,OBD,TOP1,老品,0,794,55,40,44,56,42,40,48,45.2,Ada
2,USOBDTK100,OBD,TOP10,老品,0,0,36,30,40,41,30,44,33,40,Ada
3,USCON36860B-C,CON,TOP1,老品,0,33,7,1,3,6,7,6,27,6.6,Ada
4,USCON38930B-C,CON,TOP10,老品,0,67,7,3,7,3,5,9,22,12,Ada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,USFFTHO11A0,FTK,TOP10,老品,0,0,1,1,1,1,2,0,3,1,Ada
13747,USFFTHO14A0,FTK,TOP20,老品,0,0,0,1,2,0,1,0,1,0.6,Ada
13748,USFFTTO14A0,FTK,TOP20,老品,0,0,0,1,0,1,1,3,5,3.2,Ada
13749,USFFTTO15A0,FTK,TOP20,新品,0,0,0,0,0,0,0,0,0,0,Ada


In [22]:
print(df_mrp.columns)
print(df_price.columns)

df_mrp = pd.merge(left=df_mrp,
                  right=df_price,
                  how='left',
                  left_on='SKU',
                  right_on='SKU').fillna('')

df_mrp['row'] = ['' if row == '' else int(row) for row in df_mrp['row'].tolist()]

df_mrp.to_excel('./output/3.mrp.xlsx', index=False)

df_mrp

Index(['SKU', '品类', '层级', '身份', '未售出周数', '在途总数', 'W7', 'W6', 'W5', 'W4', 'W3',
       'W2', 'W1', 'MRP周销', '品类负责人'],
      dtype='object')
Index(['主料', 'SKU', 'row', '在库', '在途', '组合在库', '组合在途'], dtype='object')


,SKU,品类,层级,身份,未售出周数,在途总数,W7,W6,W5,W4,...,W2,W1,MRP周销,品类负责人,主料,row,在库,在途,组合在库,组合在途
0,USTCV17215HIE-2-C,TCV,TOP1,老品,0,195,63,38,41,45,...,38,60,45.5,Ada,USTCV17215HIE-2-C,734,512.0,178.0,0.0,0.0
1,USOBDAD100-C,OBD,TOP1,老品,0,794,55,40,44,56,...,40,48,45.2,Ada,USOBDAD100-C,197127,81.0,717.0,0.0,0.0
2,USOBDTK100,OBD,TOP10,老品,0,0,36,30,40,41,...,44,33,40,Ada,USOBDTK100,197135,38.0,0.0,0.0,0.0
3,USCON36860B-C,CON,TOP1,老品,0,33,7,1,3,6,...,6,27,6.6,Ada,USCON36860-C,4511,95.0,8.0,0.0,0.0
4,USCON38930B-C,CON,TOP10,老品,0,67,7,3,7,3,...,9,22,12,Ada,USCON38930-C,4595,35.0,49.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,USFFTHO11A0,FTK,TOP10,老品,0,0,1,1,1,1,...,0,3,1,Ada,USFFTHO11A0,184148,4.0,0.0,0.0,0.0
13747,USFFTHO14A0,FTK,TOP20,老品,0,0,0,1,2,0,...,0,1,0.6,Ada,USFFTHO14A0,184149,6.0,0.0,0.0,0.0
13748,USFFTTO14A0,FTK,TOP20,老品,0,0,0,1,0,1,...,3,5,3.2,Ada,USFFTTO14A0,184160,7.0,0.0,0.0,0.0
13749,USFFTTO15A0,FTK,TOP20,新品,0,0,0,0,0,0,...,0,0,0,Ada,USFFTTO15A0,184161,6.0,0.0,0.0,0.0


In [23]:
df_temp = df_mrp.copy()
df_temp['times_row'] = [df_temp['row'].tolist().count(row) for row in df_temp['row'].tolist()]
df_temp['times_sku'] = [df_temp['SKU'].tolist().count(sku) for sku in df_temp['SKU'].tolist()]

df_temp

,SKU,品类,层级,身份,未售出周数,在途总数,W7,W6,W5,W4,...,MRP周销,品类负责人,主料,row,在库,在途,组合在库,组合在途,times_row,times_sku
0,USTCV17215HIE-2-C,TCV,TOP1,老品,0,195,63,38,41,45,...,45.5,Ada,USTCV17215HIE-2-C,734,512.0,178.0,0.0,0.0,1,1
1,USOBDAD100-C,OBD,TOP1,老品,0,794,55,40,44,56,...,45.2,Ada,USOBDAD100-C,197127,81.0,717.0,0.0,0.0,1,1
2,USOBDTK100,OBD,TOP10,老品,0,0,36,30,40,41,...,40,Ada,USOBDTK100,197135,38.0,0.0,0.0,0.0,1,1
3,USCON36860B-C,CON,TOP1,老品,0,33,7,1,3,6,...,6.6,Ada,USCON36860-C,4511,95.0,8.0,0.0,0.0,1,1
4,USCON38930B-C,CON,TOP10,老品,0,67,7,3,7,3,...,12,Ada,USCON38930-C,4595,35.0,49.0,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,USFFTHO11A0,FTK,TOP10,老品,0,0,1,1,1,1,...,1,Ada,USFFTHO11A0,184148,4.0,0.0,0.0,0.0,1,1
13747,USFFTHO14A0,FTK,TOP20,老品,0,0,0,1,2,0,...,0.6,Ada,USFFTHO14A0,184149,6.0,0.0,0.0,0.0,1,1
13748,USFFTTO14A0,FTK,TOP20,老品,0,0,0,1,0,1,...,3.2,Ada,USFFTTO14A0,184160,7.0,0.0,0.0,0.0,1,1
13749,USFFTTO15A0,FTK,TOP20,新品,0,0,0,0,0,0,...,0,Ada,USFFTTO15A0,184161,6.0,0.0,0.0,0.0,1,1


In [24]:
df_temp_1 = df_temp[(df_temp['row'] != '') & (df_temp['times_row'] != 1)][['SKU', 'row', 'times_row']]
df_temp_2 = df_temp[(df_temp['row'] == '') & (df_temp['times_sku'] != 1)][['SKU', 'row', 'times_sku']]
df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

if len(df_temp_1) != 0:
    df_temp_1 = df_temp_1.sort_values(by=['row', 'SKU'], ascending=[True, True]).reset_index(drop=True)
    df_temp_1.to_excel('./output/error-1.mrp重复出现-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_1

,SKU,row,times_row,times_sku


In [25]:
import sys

In [26]:
if len(df_temp_1) != 0:
    sys.exit()

In [27]:
df_temp_2 = df_temp[df_temp['row'] == ''][['SKU']]

if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2.to_excel('./output/warning-2.mrp缺失信息-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

,SKU
0,USGS01064-1-C
1,USGS46700
2,USGS49520LR
3,USGS49650
4,USGS64080
5,USGSM1048


# 4. last与MRP整合

In [28]:
df_last = pd.read_excel(last,
                        header=1,
                        usecols='A, C:L, AD:AG, AJ:AO, AQ:AU, AX:AY',
                        dtype=str).fillna('')

df_last['上次序号'] = [int(i.strip()) for i in df_last['序号'].tolist()]
df_last['Date_Left'] = df_last['当日库存'].tolist()

print(df_last.columns)

df_last

Index([                      '序号',                    '在线SKU',
                            'SKU',                 '每周一大账号售价',
                           '第一涨价',                     '第二涨价',
                           '第三涨价',                     '第四涨价',
                           '第五涨价',                     '第六涨价',
                           '第七涨价',                     '当日库存',
              2024-04-28 00:00:00,        2024-04-26 00:00:00,
              2024-04-25 00:00:00, 2024-04-23 00:00:00.100000,
          '2024-04-24 00:00:00.1',    '2024-04-25 00:00:00.1',
          '2024-04-26 00:00:00.1',        2024-04-27 00:00:00,
          '2024-04-28 00:00:00.1',                     '周销干预',
                           '改价记录',                   '本周改价记录',
                         '最新断货时间',                    '品类负责人',
                           '干预记录',                       '提价',
                           '上次序号',                'Date_Left'],
      dtype='object')


,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,2024-04-28 00:00:00.1,周销干预,改价记录,本周改价记录,最新断货时间,品类负责人,干预记录,提价,上次序号,Date_Left
0,1,USACB00278,USACB00278,87.99,,,,,,,...,0,,4.29-T2; 7.5-T3; 7.7-T3; 7.26-T10; 8.9-H; 9.24...,,2023/05/18,Ada,,,1,27
1,2,USACC68322,USACC68322-C,,60,15,,,,,...,1,,5.17-H; 5.30-T3; 6.2-T1; 6.3-T3; 6.5-改0; 6.6-改...,,2023/08/08,Gordon,,,2,46
2,3,USACC78321,USACC78321-C,,,,,,,,...,0,,6.13-T3; 6.16-T10; 6.18-T15; 7.14-改0; 7.28-T10...,,2022/07/14,Gordon,,,3,74
3,4,USACC1G010,USACC1G010-C,,10,,,,,,...,0,,9.8-T10; 3.20-J8; 4.3-H;,,,Gordon,,,4,10
4,5,USACC98442,USACC98442-C,,,,,,,,...,0,,6.1-H; 6.15-T3; 6.17-T10; 6.30-T50; 7.12-改0; 9...,,2022/07/12,Gordon,,,5,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,13747,USGS49520LR,USGS49520LR,,,,,,,,...,0,,,,,Ada,,,13747,
13747,13748,USGS49650,USGS49650,30.69,,,,,,,...,0,,,,,Ada,,,13748,
13748,13749,USGS64080,USGS64080,,,,,,,,...,0,,,,,Ada,,,13749,
13749,13750,USGSM1048,USGSM1048,,,,,,,,...,0,,,,,Ada,,,13750,


In [29]:
print(df_last.columns)
print(df_price.columns)

df_last = pd.merge(left=df_last,
                   right=df_price,
                   how='left',
                   left_on='SKU',
                   right_on='SKU').fillna('')

df_last['row'] = ['' if row == '' else int(row) for row in df_last['row'].tolist()]

df_last

Index([                      '序号',                    '在线SKU',
                            'SKU',                 '每周一大账号售价',
                           '第一涨价',                     '第二涨价',
                           '第三涨价',                     '第四涨价',
                           '第五涨价',                     '第六涨价',
                           '第七涨价',                     '当日库存',
              2024-04-28 00:00:00,        2024-04-26 00:00:00,
              2024-04-25 00:00:00, 2024-04-23 00:00:00.100000,
          '2024-04-24 00:00:00.1',    '2024-04-25 00:00:00.1',
          '2024-04-26 00:00:00.1',        2024-04-27 00:00:00,
          '2024-04-28 00:00:00.1',                     '周销干预',
                           '改价记录',                   '本周改价记录',
                         '最新断货时间',                    '品类负责人',
                           '干预记录',                       '提价',
                           '上次序号',                'Date_Left'],
      dtype='object')
Index(['主料', 'SKU', 'row', '在库',

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,干预记录,提价,上次序号,Date_Left,主料,row,在库,在途,组合在库,组合在途
0,1,USACB00278,USACB00278,87.99,,,,,,,...,,,1,27,USACB00278,139824,27.0,0.0,0.0,0.0
1,2,USACC68322,USACC68322-C,,60,15,,,,,...,,,2,46,USACC68322-C,3135,45.0,41.0,0.0,0.0
2,3,USACC78321,USACC78321-C,,,,,,,,...,,,3,74,USACC78321-C,3924,74.0,0.0,0.0,0.0
3,4,USACC1G010,USACC1G010-C,,10,,,,,,...,,,4,10,USACC1G010-C,5351,10.0,2.0,0.0,0.0
4,5,USACC98442,USACC98442-C,,,,,,,,...,,,5,20,USACC98442-C,4898,20.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,13747,USGS49520LR,USGS49520LR,,,,,,,,...,,,13747,,,,,,,
13747,13748,USGS49650,USGS49650,30.69,,,,,,,...,,,13748,,,,,,,
13748,13749,USGS64080,USGS64080,,,,,,,,...,,,13749,,,,,,,
13749,13750,USGSM1048,USGSM1048,,,,,,,,...,,,13750,,,,,,,


In [30]:
df_temp = df_last.copy()
df_temp['times_row'] = [df_temp['row'].tolist().count(row) for row in df_temp['row'].tolist()]
df_temp['times_sku'] = [df_temp['SKU'].tolist().count(sku) for sku in df_temp['SKU'].tolist()]

df_temp

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,上次序号,Date_Left,主料,row,在库,在途,组合在库,组合在途,times_row,times_sku
0,1,USACB00278,USACB00278,87.99,,,,,,,...,1,27,USACB00278,139824,27.0,0.0,0.0,0.0,1,1
1,2,USACC68322,USACC68322-C,,60,15,,,,,...,2,46,USACC68322-C,3135,45.0,41.0,0.0,0.0,1,1
2,3,USACC78321,USACC78321-C,,,,,,,,...,3,74,USACC78321-C,3924,74.0,0.0,0.0,0.0,1,1
3,4,USACC1G010,USACC1G010-C,,10,,,,,,...,4,10,USACC1G010-C,5351,10.0,2.0,0.0,0.0,1,1
4,5,USACC98442,USACC98442-C,,,,,,,,...,5,20,USACC98442-C,4898,20.0,0.0,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,13747,USGS49520LR,USGS49520LR,,,,,,,,...,13747,,,,,,,,6,1
13747,13748,USGS49650,USGS49650,30.69,,,,,,,...,13748,,,,,,,,6,1
13748,13749,USGS64080,USGS64080,,,,,,,,...,13749,,,,,,,,6,1
13749,13750,USGSM1048,USGSM1048,,,,,,,,...,13750,,,,,,,,6,1


In [31]:
df_temp_1 = df_temp[(df_temp['row'] != '') & (df_temp['times_row'] != 1)][['SKU', 'row', 'times_row']]
df_temp_2 = df_temp[(df_temp['row'] == '') & (df_temp['times_sku'] != 1)][['SKU', 'row', 'times_sku']]
df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

if len(df_temp_1) != 0:
    df_temp_1 = df_temp_1.sort_values(by=['row', 'SKU'], ascending=[True, True]).reset_index(drop=True)
    df_temp_1.to_excel('./output/error-2.last重复出现-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_1

,SKU,row,times_row,times_sku


In [32]:
if len(df_temp_1) != 0:
    sys.exit()

In [33]:
df_temp_2 = df_temp[df_temp['row'] == ''][['SKU']]

if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2.to_excel('./output/warning-3.last缺失信息-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

,SKU
0,USGS01064-1-C
1,USGS46700
2,USGS49520LR
3,USGS49650
4,USGS64080
5,USGSM1048


In [34]:
print(df_last.columns)
print(df_mrp.columns)

# 找出有库存信息的数据
df_temp = pd.merge(left=df_last[df_last['row'] != ''],
                   right=df_mrp,
                   how='left',
                   left_on='row',
                   right_on='row',
                   suffixes=('', '_mrp')).fillna('')

df_temp['SKU'] = [sku if sku_mrp == '' else sku_mrp for sku, sku_mrp in zip(df_temp['SKU'].tolist(), df_temp['SKU_mrp'].tolist())]
df_temp['Note'] = ['' if charge == 'Ada' else 'last only, not in MRP' for charge in df_temp['品类负责人_mrp'].tolist()]
list_column = df_temp.columns.tolist()
list_column.remove('SKU_mrp')
list_column.remove('主料_mrp')
list_column.remove('在库_mrp')
list_column.remove('在途_mrp')
list_column.remove('组合在库_mrp')
list_column.remove('组合在途_mrp')
df_temp = df_temp[list_column]

df_temp

Index([                      '序号',                    '在线SKU',
                            'SKU',                 '每周一大账号售价',
                           '第一涨价',                     '第二涨价',
                           '第三涨价',                     '第四涨价',
                           '第五涨价',                     '第六涨价',
                           '第七涨价',                     '当日库存',
              2024-04-28 00:00:00,        2024-04-26 00:00:00,
              2024-04-25 00:00:00, 2024-04-23 00:00:00.100000,
          '2024-04-24 00:00:00.1',    '2024-04-25 00:00:00.1',
          '2024-04-26 00:00:00.1',        2024-04-27 00:00:00,
          '2024-04-28 00:00:00.1',                     '周销干预',
                           '改价记录',                   '本周改价记录',
                         '最新断货时间',                    '品类负责人',
                           '干预记录',                       '提价',
                           '上次序号',                'Date_Left',
                             '主料',                     

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,W7,W6,W5,W4,W3,W2,W1,MRP周销,品类负责人_mrp,Note
0,1,USACB00278,USACB00278,87.99,,,,,,,...,0,0,0,0,0,0,0,0,Ada,
1,2,USACC68322,USACC68322-C,,60,15,,,,,...,6,3,0,5,1,4,4,3.58,Ada,
2,3,USACC78321,USACC78321-C,,,,,,,,...,2,2,1,2,0,2,4,4.49,Ada,
3,4,USACC1G010,USACC1G010-C,,10,,,,,,...,0,0,0,1,0,0,0,0.51,Ada,
4,5,USACC98442,USACC98442-C,,,,,,,,...,0,0,1,0,0,0,0,0.5,Ada,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13740,13741,,USFP22590-C,,,,,,,,...,0,0,0,0,0,0,0,0.5,Ada,
13741,13742,,USFP23090-C,,,,,,,,...,0,0,0,0,0,0,0,0.5,Ada,
13742,13743,,USGS05640-1-C,,,,,,,,...,0,0,0,0,0,0,0,0.5,Ada,
13743,13744,,USGS12407-2-C,,,,,,,,...,0,0,0,0,0,0,0,0.25,Ada,


In [35]:
print(df_last.columns)
print(df_mrp.columns)

# 找出没有库存信息的数据
df_temp_2 = pd.merge(left=df_last[df_last['row'] == ''],
                     right=df_mrp,
                     how='left',
                     left_on='SKU',
                     right_on='SKU',
                     suffixes=('', '_mrp')).fillna('')

df_temp_2['在库'] = [stock if stock_mrp == '' else stock_mrp for stock, stock_mrp in zip(df_temp_2['在库'].tolist(), df_temp_2['在库_mrp'].tolist())]
df_temp_2['在途'] = [way if way_mrp == '' else way_mrp for way, way_mrp in zip(df_temp_2['在途'].tolist(), df_temp_2['在途_mrp'].tolist())]
df_temp_2['组合在库'] = [stock if stock_mrp == '' else stock_mrp for stock, stock_mrp in zip(df_temp_2['组合在库'].tolist(), df_temp_2['组合在库_mrp'].tolist())]
df_temp_2['组合在途'] = [way if way_mrp == '' else way_mrp for way, way_mrp in zip(df_temp_2['组合在途'].tolist(), df_temp_2['组合在途_mrp'].tolist())]
df_temp_2['Note'] = ['' if charge == 'Ada' else 'last only, not in MRP' for charge in df_temp_2['品类负责人_mrp'].tolist()]
list_column = df_temp_2.columns.tolist()
list_column.remove('品类负责人_mrp')
list_column.remove('主料_mrp')
list_column.remove('row_mrp')
list_column.remove('在库_mrp')
list_column.remove('在途_mrp')
list_column.remove('组合在库_mrp')
list_column.remove('组合在途_mrp')
df_temp_2 = df_temp_2[list_column]

df_temp_2

Index([                      '序号',                    '在线SKU',
                            'SKU',                 '每周一大账号售价',
                           '第一涨价',                     '第二涨价',
                           '第三涨价',                     '第四涨价',
                           '第五涨价',                     '第六涨价',
                           '第七涨价',                     '当日库存',
              2024-04-28 00:00:00,        2024-04-26 00:00:00,
              2024-04-25 00:00:00, 2024-04-23 00:00:00.100000,
          '2024-04-24 00:00:00.1',    '2024-04-25 00:00:00.1',
          '2024-04-26 00:00:00.1',        2024-04-27 00:00:00,
          '2024-04-28 00:00:00.1',                     '周销干预',
                           '改价记录',                   '本周改价记录',
                         '最新断货时间',                    '品类负责人',
                           '干预记录',                       '提价',
                           '上次序号',                'Date_Left',
                             '主料',                     

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,在途总数,W7,W6,W5,W4,W3,W2,W1,MRP周销,Note
0,13746,USGS46700,USGS46700,,,,,,,,...,0,0,0,0,0,0,0,0,0,
1,13747,USGS49520LR,USGS49520LR,,,,,,,,...,0,0,0,0,0,0,0,0,0,
2,13748,USGS49650,USGS49650,30.69,,,,,,,...,0,0,0,0,0,0,0,0,0,
3,13749,USGS64080,USGS64080,,,,,,,,...,0,0,0,0,0,0,0,0,0,
4,13750,USGSM1048,USGSM1048,,,,,,,,...,0,0,0,0,0,0,0,0,0,
5,13751,,USGS01064-1-C,,,,,,,,...,0,0,0,0,0,0,0,0,0,


In [36]:
# 整合数据
df_temp = pd.concat([df_temp, df_temp_2], ignore_index=True)

df_temp

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,W7,W6,W5,W4,W3,W2,W1,MRP周销,品类负责人_mrp,Note
0,1,USACB00278,USACB00278,87.99,,,,,,,...,0,0,0,0,0,0,0,0,Ada,
1,2,USACC68322,USACC68322-C,,60,15,,,,,...,6,3,0,5,1,4,4,3.58,Ada,
2,3,USACC78321,USACC78321-C,,,,,,,,...,2,2,1,2,0,2,4,4.49,Ada,
3,4,USACC1G010,USACC1G010-C,,10,,,,,,...,0,0,0,1,0,0,0,0.51,Ada,
4,5,USACC98442,USACC98442-C,,,,,,,,...,0,0,1,0,0,0,0,0.5,Ada,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,13747,USGS49520LR,USGS49520LR,,,,,,,,...,0,0,0,0,0,0,0,0,NaN,
13747,13748,USGS49650,USGS49650,30.69,,,,,,,...,0,0,0,0,0,0,0,0,NaN,
13748,13749,USGS64080,USGS64080,,,,,,,,...,0,0,0,0,0,0,0,0,NaN,
13749,13750,USGSM1048,USGSM1048,,,,,,,,...,0,0,0,0,0,0,0,0,NaN,


In [37]:
print(df_mrp.columns)
print(df_last.columns)

df_temp_2 = pd.merge(left=df_mrp,
                     right=df_last,
                     how='left',
                     left_on='SKU',
                     right_on='SKU',
                     suffixes=('', '_last')).fillna('')

df_temp_2 = df_temp_2[df_temp_2['序号'] == ''].reset_index(drop=True)
for i in range(len(df_temp_2)):
    if df_temp_2.loc[i, 'SKU'] not in df_temp['SKU'].tolist():
        df_temp_2.loc[i, 'Note'] = 'MRP only, not in last'

if len(df_temp_2) > 0:
    df_temp_2 = df_temp_2[df_temp_2['Note'] == 'MRP only, not in last'].reset_index(drop=True)

df_temp_2['品类负责人'] = df_temp_2['品类负责人_last'].tolist()
    
list_column = df_temp_2.columns.tolist()
list_column.remove('品类负责人_last')
list_column.remove('主料_last')
list_column.remove('row_last')
list_column.remove('在库_last')
list_column.remove('在途_last')
list_column.remove('组合在库_last')
list_column.remove('组合在途_last')
df_temp_2 = df_temp_2[list_column]

df_temp_2

Index(['SKU', '品类', '层级', '身份', '未售出周数', '在途总数', 'W7', 'W6', 'W5', 'W4', 'W3',
       'W2', 'W1', 'MRP周销', '品类负责人', '主料', 'row', '在库', '在途', '组合在库', '组合在途'],
      dtype='object')
Index([                      '序号',                    '在线SKU',
                            'SKU',                 '每周一大账号售价',
                           '第一涨价',                     '第二涨价',
                           '第三涨价',                     '第四涨价',
                           '第五涨价',                     '第六涨价',
                           '第七涨价',                     '当日库存',
              2024-04-28 00:00:00,        2024-04-26 00:00:00,
              2024-04-25 00:00:00, 2024-04-23 00:00:00.100000,
          '2024-04-24 00:00:00.1',    '2024-04-25 00:00:00.1',
          '2024-04-26 00:00:00.1',        2024-04-27 00:00:00,
          '2024-04-28 00:00:00.1',                     '周销干预',
                           '改价记录',                   '本周改价记录',
                         '最新断货时间',                    '品类负责人',
 

,SKU,品类,层级,身份,未售出周数,在途总数,W7,W6,W5,W4,...,2024-04-27 00:00:00,2024-04-28 00:00:00.1,周销干预,改价记录,本周改价记录,最新断货时间,干预记录,提价,上次序号,Date_Left


In [38]:
df = pd.concat([df_temp, df_temp_2], ignore_index=True)

df

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,W7,W6,W5,W4,W3,W2,W1,MRP周销,品类负责人_mrp,Note
0,1,USACB00278,USACB00278,87.99,,,,,,,...,0,0,0,0,0,0,0,0,Ada,
1,2,USACC68322,USACC68322-C,,60,15,,,,,...,6,3,0,5,1,4,4,3.58,Ada,
2,3,USACC78321,USACC78321-C,,,,,,,,...,2,2,1,2,0,2,4,4.49,Ada,
3,4,USACC1G010,USACC1G010-C,,10,,,,,,...,0,0,0,1,0,0,0,0.51,Ada,
4,5,USACC98442,USACC98442-C,,,,,,,,...,0,0,1,0,0,0,0,0.5,Ada,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,13747,USGS49520LR,USGS49520LR,,,,,,,,...,0,0,0,0,0,0,0,0,NaN,
13747,13748,USGS49650,USGS49650,30.69,,,,,,,...,0,0,0,0,0,0,0,0,NaN,
13748,13749,USGS64080,USGS64080,,,,,,,,...,0,0,0,0,0,0,0,0,NaN,
13749,13750,USGSM1048,USGSM1048,,,,,,,,...,0,0,0,0,0,0,0,0,NaN,


# 5. 其他计算

In [39]:
df_missing = pd.read_excel(price,
                           sheet_name=1,
                           header=0,
                           names=['SKU', '数量'],
                           usecols='D, F',
                           dtype=str).fillna('')

df_missing = df_missing[df_missing['SKU'] != ''].reset_index(drop=True)

df_missing['数量'] = df_missing['数量'].astype(int)
df_missing = df_missing.groupby([df_missing['SKU']], as_index=False).sum()

df_missing

,SKU,数量
0,USABS10000-C,1
1,USABS10010-C,1
2,USABS10130-C,1
3,USABS1370-C,1
4,USABS13980-C,1
...,...,...
1758,USWSP4TD01B0-4-YC,1
1759,USWSP54525B1-2-YC,1
1760,USWSP5TA04B1-4-YC,1
1761,USWSP5TN01B0-2-YC,1


In [40]:
print(df_missing.columns)
print(df_price.columns)

df_missing = pd.merge(left=df_missing,
                      right=df_price,
                      how='left',
                      left_on='SKU',
                      right_on='SKU').fillna('')

df_missing

Index(['SKU', '数量'], dtype='object')
Index(['主料', 'SKU', 'row', '在库', '在途', '组合在库', '组合在途'], dtype='object')


,SKU,数量,主料,row,在库,在途,组合在库,组合在途
0,USABS10000-C,1,USABS10000-C,137217.0,2.0,4.0,0.0,0.0
1,USABS10010-C,1,USABS10010-C,137219.0,5.0,6.0,0.0,0.0
2,USABS10130-C,1,USABS10130-C,137226.0,6.0,5.0,0.0,0.0
3,USABS1370-C,1,USABS1370-C,137449.0,12.0,0.0,0.0,0.0
4,USABS13980-C,1,USABS13980-C,137463.0,10.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
1758,USWSP4TD01B0-4-YC,1,USWSP4TD01B0-4-YC,229185.0,7.0,0.0,0.0,0.0
1759,USWSP54525B1-2-YC,1,USWSP54525B1-2-YC,229272.0,10.0,6.0,0.0,0.0
1760,USWSP5TA04B1-4-YC,1,USWSP5TA04B1-4-YC,229427.0,32.0,0.0,0.0,0.0
1761,USWSP5TN01B0-2-YC,1,USWSP5TN01B0-2-YC,229468.0,19.0,0.0,0.0,0.0


In [41]:
df_temp = df_missing.copy()
df_temp['times'] = [df_temp['row'].tolist().count(row) for row in df_temp['row'].tolist()]

df_temp

,SKU,数量,主料,row,在库,在途,组合在库,组合在途,times
0,USABS10000-C,1,USABS10000-C,137217.0,2.0,4.0,0.0,0.0,1
1,USABS10010-C,1,USABS10010-C,137219.0,5.0,6.0,0.0,0.0,1
2,USABS10130-C,1,USABS10130-C,137226.0,6.0,5.0,0.0,0.0,1
3,USABS1370-C,1,USABS1370-C,137449.0,12.0,0.0,0.0,0.0,1
4,USABS13980-C,1,USABS13980-C,137463.0,10.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...
1758,USWSP4TD01B0-4-YC,1,USWSP4TD01B0-4-YC,229185.0,7.0,0.0,0.0,0.0,1
1759,USWSP54525B1-2-YC,1,USWSP54525B1-2-YC,229272.0,10.0,6.0,0.0,0.0,1
1760,USWSP5TA04B1-4-YC,1,USWSP5TA04B1-4-YC,229427.0,32.0,0.0,0.0,0.0,1
1761,USWSP5TN01B0-2-YC,1,USWSP5TN01B0-2-YC,229468.0,19.0,0.0,0.0,0.0,1


In [42]:
# 找出有库存信息的数据
df_temp_1 = df_temp[(df_temp['row'] != '') & (df_temp['times'] != 1)][['row', '数量']]
df_temp_1 = df_temp_1.groupby([df_temp_1['row']], as_index=False).sum()
df_temp_2 = df_temp[(df_temp['row'] != '') & (df_temp['times']) == 1][['row', '数量']]

df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

df_temp_1

,row,数量
0,137217.0,1
1,137219.0,1
2,137226.0,1
3,137449.0,1
4,137463.0,1
...,...,...
1603,229185.0,1
1604,229272.0,1
1605,229427.0,1
1606,229468.0,1


In [43]:
print(df.columns)
print(df_temp_1.columns)

df = pd.merge(left=df,
              right=df_temp_1,
              how='left',
              left_on='row',
              right_on='row').fillna(0)

df

Index([                      '序号',                    '在线SKU',
                            'SKU',                 '每周一大账号售价',
                           '第一涨价',                     '第二涨价',
                           '第三涨价',                     '第四涨价',
                           '第五涨价',                     '第六涨价',
                           '第七涨价',                     '当日库存',
              2024-04-28 00:00:00,        2024-04-26 00:00:00,
              2024-04-25 00:00:00, 2024-04-23 00:00:00.100000,
          '2024-04-24 00:00:00.1',    '2024-04-25 00:00:00.1',
          '2024-04-26 00:00:00.1',        2024-04-27 00:00:00,
          '2024-04-28 00:00:00.1',                     '周销干预',
                           '改价记录',                   '本周改价记录',
                         '最新断货时间',                    '品类负责人',
                           '干预记录',                       '提价',
                           '上次序号',                'Date_Left',
                             '主料',                     

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,W6,W5,W4,W3,W2,W1,MRP周销,品类负责人_mrp,Note,数量
0,1,USACB00278,USACB00278,87.99,,,,,,,...,0,0,0,0,0,0,0,Ada,,0.0
1,2,USACC68322,USACC68322-C,,60,15,,,,,...,3,0,5,1,4,4,3.58,Ada,,0.0
2,3,USACC78321,USACC78321-C,,,,,,,,...,2,1,2,0,2,4,4.49,Ada,,0.0
3,4,USACC1G010,USACC1G010-C,,10,,,,,,...,0,0,1,0,0,0,0.51,Ada,,0.0
4,5,USACC98442,USACC98442-C,,,,,,,,...,0,1,0,0,0,0,0.5,Ada,,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,13747,USGS49520LR,USGS49520LR,,,,,,,,...,0,0,0,0,0,0,0,0,,0.0
13747,13748,USGS49650,USGS49650,30.69,,,,,,,...,0,0,0,0,0,0,0,0,,0.0
13748,13749,USGS64080,USGS64080,,,,,,,,...,0,0,0,0,0,0,0,0,,0.0
13749,13750,USGSM1048,USGSM1048,,,,,,,,...,0,0,0,0,0,0,0,0,,0.0


In [44]:
# 找出没有库存信息的数据
df_temp_2 = df_temp[df_temp['row'] == ''][['SKU', '数量']]
                                          
if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2[['SKU']].to_excel('./output/warning-4.无record缺失信息-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

,SKU,数量
0,USABS26890-C,1
1,USABS47900A0-C,1
2,USAIH96053U-C,1
3,USAIH96090U-C,1
4,USAIH96095U-C,1
...,...,...
150,USWRM41181-C,1
151,USWRM41727-C,1
152,USWRM41728-C,1
153,USWS02535U-C,1


In [45]:
print(df.columns)
print(df_temp_2.columns)

df = pd.merge(left=df,
              right=df_temp_2,
              how='left',
              left_on='SKU',
              right_on='SKU',
              suffixes=('_1', '_2')).fillna(0)

df

Index([                      '序号',                    '在线SKU',
                            'SKU',                 '每周一大账号售价',
                           '第一涨价',                     '第二涨价',
                           '第三涨价',                     '第四涨价',
                           '第五涨价',                     '第六涨价',
                           '第七涨价',                     '当日库存',
              2024-04-28 00:00:00,        2024-04-26 00:00:00,
              2024-04-25 00:00:00, 2024-04-23 00:00:00.100000,
          '2024-04-24 00:00:00.1',    '2024-04-25 00:00:00.1',
          '2024-04-26 00:00:00.1',        2024-04-27 00:00:00,
          '2024-04-28 00:00:00.1',                     '周销干预',
                           '改价记录',                   '本周改价记录',
                         '最新断货时间',                    '品类负责人',
                           '干预记录',                       '提价',
                           '上次序号',                'Date_Left',
                             '主料',                     

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,W5,W4,W3,W2,W1,MRP周销,品类负责人_mrp,Note,数量_1,数量_2
0,1,USACB00278,USACB00278,87.99,,,,,,,...,0,0,0,0,0,0,Ada,,0.0,0.0
1,2,USACC68322,USACC68322-C,,60,15,,,,,...,0,5,1,4,4,3.58,Ada,,0.0,0.0
2,3,USACC78321,USACC78321-C,,,,,,,,...,1,2,0,2,4,4.49,Ada,,0.0,0.0
3,4,USACC1G010,USACC1G010-C,,10,,,,,,...,0,1,0,0,0,0.51,Ada,,0.0,0.0
4,5,USACC98442,USACC98442-C,,,,,,,,...,1,0,0,0,0,0.5,Ada,,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,13747,USGS49520LR,USGS49520LR,,,,,,,,...,0,0,0,0,0,0,0,,0.0,0.0
13747,13748,USGS49650,USGS49650,30.69,,,,,,,...,0,0,0,0,0,0,0,,0.0,0.0
13748,13749,USGS64080,USGS64080,,,,,,,,...,0,0,0,0,0,0,0,,0.0,0.0
13749,13750,USGSM1048,USGSM1048,,,,,,,,...,0,0,0,0,0,0,0,,0.0,0.0


In [46]:
df['无Record ID'] = [amount_1 + amount_2 for amount_1, amount_2 in zip(df['数量_1'].tolist(), df['数量_2'].tolist())]
df['在库总数'] = [stock_1 + stock_2 for stock_1, stock_2 in zip(df['在库'].tolist(), df['组合在库'].tolist())]
df['在途总数'] = [way_1 + way_2 for way_1, way_2 in zip(df['在途'].tolist(), df['组合在途'].tolist())]

list_column = df.columns.tolist()
list_column.remove('主料')
list_column.remove('在库')
list_column.remove('组合在库')
list_column.remove('在途')
list_column.remove('组合在途')
list_column.remove('数量_1')
list_column.remove('数量_2')
df = df[list_column]

df

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,W5,W4,W3,W2,W1,MRP周销,品类负责人_mrp,Note,无Record ID,在库总数
0,1,USACB00278,USACB00278,87.99,,,,,,,...,0,0,0,0,0,0,Ada,,0.0,27.0
1,2,USACC68322,USACC68322-C,,60,15,,,,,...,0,5,1,4,4,3.58,Ada,,0.0,45.0
2,3,USACC78321,USACC78321-C,,,,,,,,...,1,2,0,2,4,4.49,Ada,,0.0,74.0
3,4,USACC1G010,USACC1G010-C,,10,,,,,,...,0,1,0,0,0,0.51,Ada,,0.0,10.0
4,5,USACC98442,USACC98442-C,,,,,,,,...,1,0,0,0,0,0.5,Ada,,0.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,13747,USGS49520LR,USGS49520LR,,,,,,,,...,0,0,0,0,0,0,0,,0.0,
13747,13748,USGS49650,USGS49650,30.69,,,,,,,...,0,0,0,0,0,0,0,,0.0,
13748,13749,USGS64080,USGS64080,,,,,,,,...,0,0,0,0,0,0,0,,0.0,
13749,13750,USGSM1048,USGSM1048,,,,,,,,...,0,0,0,0,0,0,0,,0.0,


In [47]:
df_sale = pd.read_excel(price,
                        sheet_name=2,
                        header=0,
                        names=['SKU', '数量'],
                        usecols='D, F',
                        dtype=str).fillna('')

df_sale = df_sale[df_sale['SKU'] != ''].reset_index(drop=True)

df_sale['数量'] = df_sale['数量'].astype(int)
df_sale = df_sale.groupby(df_sale['SKU'], as_index=False).sum()

df_sale

,SKU,数量
0,USABS10130-C,1
1,USABS1020-C,1
2,USABS10380-C,1
3,USABS10870-C,1
4,USABS11430-C,1
...,...,...
2059,USWSP55101B1-4-YC,1
2060,USWSP55301B1-4-YC,1
2061,USWSP5TF04B0-4-YC,1
2062,USWSP64503B1-4-YC,1


In [48]:
print(df_sale.columns)
print(df_price.columns)

df_sale = pd.merge(left=df_sale,
                   right=df_price,
                   how='left',
                   left_on='SKU',
                   right_on='SKU').fillna('')

df_sale

Index(['SKU', '数量'], dtype='object')
Index(['主料', 'SKU', 'row', '在库', '在途', '组合在库', '组合在途'], dtype='object')


,SKU,数量,主料,row,在库,在途,组合在库,组合在途
0,USABS10130-C,1,USABS10130-C,137226.0,6.0,5.0,0.0,0.0
1,USABS1020-C,1,USABS1020-C,137234.0,0.0,0.0,0.0,0.0
2,USABS10380-C,1,USABS10380-C,137250.0,6.0,2.0,0.0,0.0
3,USABS10870-C,1,USABS10870-C,137271.0,7.0,8.0,0.0,0.0
4,USABS11430-C,1,USABS11430-C,137326.0,8.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2059,USWSP55101B1-4-YC,1,USWSP55101B1-4-YC,229381.0,5.0,1.0,0.0,0.0
2060,USWSP55301B1-4-YC,1,USWSP55301B1-4-YC,229385.0,6.0,0.0,0.0,0.0
2061,USWSP5TF04B0-4-YC,1,USWSP5TF04B0-4-YC,229451.0,8.0,0.0,0.0,0.0
2062,USWSP64503B1-4-YC,1,USWSP64503B1-4-YC,229479.0,8.0,2.0,0.0,0.0


In [49]:
df_temp = df_sale.copy()
df_temp['times'] = [df_temp['row'].tolist().count(row) for row in df_temp['row'].tolist()]

df_temp

,SKU,数量,主料,row,在库,在途,组合在库,组合在途,times
0,USABS10130-C,1,USABS10130-C,137226.0,6.0,5.0,0.0,0.0,1
1,USABS1020-C,1,USABS1020-C,137234.0,0.0,0.0,0.0,0.0,1
2,USABS10380-C,1,USABS10380-C,137250.0,6.0,2.0,0.0,0.0,1
3,USABS10870-C,1,USABS10870-C,137271.0,7.0,8.0,0.0,0.0,1
4,USABS11430-C,1,USABS11430-C,137326.0,8.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...
2059,USWSP55101B1-4-YC,1,USWSP55101B1-4-YC,229381.0,5.0,1.0,0.0,0.0,1
2060,USWSP55301B1-4-YC,1,USWSP55301B1-4-YC,229385.0,6.0,0.0,0.0,0.0,1
2061,USWSP5TF04B0-4-YC,1,USWSP5TF04B0-4-YC,229451.0,8.0,0.0,0.0,0.0,1
2062,USWSP64503B1-4-YC,1,USWSP64503B1-4-YC,229479.0,8.0,2.0,0.0,0.0,1


In [50]:
# 找出有库存信息的数据
df_temp_1 = df_temp[(df_temp['row'] != '') & (df_temp['times'] != 1)][['row', '数量']]
df_temp_1 = df_temp_1.groupby([df_temp_1['row']], as_index=False).sum()
df_temp_2 = df_temp[(df_temp['row'] != '') & (df_temp['times']) == 1][['row', '数量']]

df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

df_temp_1

,row,数量
0,137226.0,1
1,137234.0,1
2,137250.0,1
3,137271.0,1
4,137326.0,1
...,...,...
1890,229381.0,1
1891,229385.0,1
1892,229451.0,1
1893,229479.0,1


In [51]:
print(df.columns)
print(df_temp_1.columns)

df = pd.merge(left=df,
              right=df_temp_1,
              how='left',
              left_on='row',
              right_on='row').fillna(0)

df

Index([                      '序号',                    '在线SKU',
                            'SKU',                 '每周一大账号售价',
                           '第一涨价',                     '第二涨价',
                           '第三涨价',                     '第四涨价',
                           '第五涨价',                     '第六涨价',
                           '第七涨价',                     '当日库存',
              2024-04-28 00:00:00,        2024-04-26 00:00:00,
              2024-04-25 00:00:00, 2024-04-23 00:00:00.100000,
          '2024-04-24 00:00:00.1',    '2024-04-25 00:00:00.1',
          '2024-04-26 00:00:00.1',        2024-04-27 00:00:00,
          '2024-04-28 00:00:00.1',                     '周销干预',
                           '改价记录',                   '本周改价记录',
                         '最新断货时间',                    '品类负责人',
                           '干预记录',                       '提价',
                           '上次序号',                'Date_Left',
                            'row',                     

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,W4,W3,W2,W1,MRP周销,品类负责人_mrp,Note,无Record ID,在库总数,数量
0,1,USACB00278,USACB00278,87.99,,,,,,,...,0,0,0,0,0,Ada,,0.0,27.0,0.0
1,2,USACC68322,USACC68322-C,,60,15,,,,,...,5,1,4,4,3.58,Ada,,0.0,45.0,1.0
2,3,USACC78321,USACC78321-C,,,,,,,,...,2,0,2,4,4.49,Ada,,0.0,74.0,1.0
3,4,USACC1G010,USACC1G010-C,,10,,,,,,...,1,0,0,0,0.51,Ada,,0.0,10.0,0.0
4,5,USACC98442,USACC98442-C,,,,,,,,...,0,0,0,0,0.5,Ada,,0.0,20.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,13747,USGS49520LR,USGS49520LR,,,,,,,,...,0,0,0,0,0,0,,0.0,,0.0
13747,13748,USGS49650,USGS49650,30.69,,,,,,,...,0,0,0,0,0,0,,0.0,,0.0
13748,13749,USGS64080,USGS64080,,,,,,,,...,0,0,0,0,0,0,,0.0,,0.0
13749,13750,USGSM1048,USGSM1048,,,,,,,,...,0,0,0,0,0,0,,0.0,,0.0


In [52]:
# 找出没有库存信息的数据
df_temp_2 = df_temp[df_temp['row'] == ''][['SKU', '数量']]
                                          
if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2[['SKU']].to_excel('./output/warning-5.sale缺失信息-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

,SKU,数量
0,USABS11540-C,1
1,USABS22660-C,1
2,USAIH62170U-C,1
3,USAIH96052U-C,1
4,USAIH96613-C,1
...,...,...
164,USWR51349-C,1
165,USWR52928H-C,1
166,USWR609FL-C,1
167,USWR7009AHY-C,1


In [53]:
print(df.columns)
print(df_temp_2.columns)

df = pd.merge(left=df,
              right=df_temp_2,
              how='left',
              left_on='SKU',
              right_on='SKU',
              suffixes=('_1', '_2')).fillna(0)

df

Index([                      '序号',                    '在线SKU',
                            'SKU',                 '每周一大账号售价',
                           '第一涨价',                     '第二涨价',
                           '第三涨价',                     '第四涨价',
                           '第五涨价',                     '第六涨价',
                           '第七涨价',                     '当日库存',
              2024-04-28 00:00:00,        2024-04-26 00:00:00,
              2024-04-25 00:00:00, 2024-04-23 00:00:00.100000,
          '2024-04-24 00:00:00.1',    '2024-04-25 00:00:00.1',
          '2024-04-26 00:00:00.1',        2024-04-27 00:00:00,
          '2024-04-28 00:00:00.1',                     '周销干预',
                           '改价记录',                   '本周改价记录',
                         '最新断货时间',                    '品类负责人',
                           '干预记录',                       '提价',
                           '上次序号',                'Date_Left',
                            'row',                     

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,W3,W2,W1,MRP周销,品类负责人_mrp,Note,无Record ID,在库总数,数量_1,数量_2
0,1,USACB00278,USACB00278,87.99,,,,,,,...,0,0,0,0,Ada,,0.0,27.0,0.0,0.0
1,2,USACC68322,USACC68322-C,,60,15,,,,,...,1,4,4,3.58,Ada,,0.0,45.0,1.0,0.0
2,3,USACC78321,USACC78321-C,,,,,,,,...,0,2,4,4.49,Ada,,0.0,74.0,1.0,0.0
3,4,USACC1G010,USACC1G010-C,,10,,,,,,...,0,0,0,0.51,Ada,,0.0,10.0,0.0,0.0
4,5,USACC98442,USACC98442-C,,,,,,,,...,0,0,0,0.5,Ada,,0.0,20.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,13747,USGS49520LR,USGS49520LR,,,,,,,,...,0,0,0,0,0,,0.0,,0.0,0.0
13747,13748,USGS49650,USGS49650,30.69,,,,,,,...,0,0,0,0,0,,0.0,,0.0,0.0
13748,13749,USGS64080,USGS64080,,,,,,,,...,0,0,0,0,0,,0.0,,0.0,0.0
13749,13750,USGSM1048,USGSM1048,,,,,,,,...,0,0,0,0,0,,0.0,,0.0,0.0


In [54]:
df['Date_Right'] = [amount_1 + amount_2 for amount_1, amount_2 in zip(df['数量_1'].tolist(), df['数量_2'].tolist())]

list_column = df.columns.tolist()
list_column.remove('row')
list_column.remove('数量_1')
list_column.remove('数量_2')
df = df[list_column]

df

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,W4,W3,W2,W1,MRP周销,品类负责人_mrp,Note,无Record ID,在库总数,Date_Right
0,1,USACB00278,USACB00278,87.99,,,,,,,...,0,0,0,0,0,Ada,,0.0,27.0,0.0
1,2,USACC68322,USACC68322-C,,60,15,,,,,...,5,1,4,4,3.58,Ada,,0.0,45.0,1.0
2,3,USACC78321,USACC78321-C,,,,,,,,...,2,0,2,4,4.49,Ada,,0.0,74.0,1.0
3,4,USACC1G010,USACC1G010-C,,10,,,,,,...,1,0,0,0,0.51,Ada,,0.0,10.0,0.0
4,5,USACC98442,USACC98442-C,,,,,,,,...,0,0,0,0,0.5,Ada,,0.0,20.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,13747,USGS49520LR,USGS49520LR,,,,,,,,...,0,0,0,0,0,0,,0.0,,0.0
13747,13748,USGS49650,USGS49650,30.69,,,,,,,...,0,0,0,0,0,0,,0.0,,0.0
13748,13749,USGS64080,USGS64080,,,,,,,,...,0,0,0,0,0,0,,0.0,,0.0
13749,13750,USGSM1048,USGSM1048,,,,,,,,...,0,0,0,0,0,0,,0.0,,0.0


In [55]:
df['累计涨价'] = ['=SUM(F' + str(i+3) + ':L' + str(i+3) + ')' for i in range(len(df))]
df['近7周平均销量'] = ['=IFERROR(ROUNDUP(AVERAGE(S' + str(i+3) + ':Y' + str(i+3) + '),1),"-")' for i in range(len(df))]
df['改价'] = ['=IFERROR(IF(AC' + str(i+3) + '-AE' + str(i+3) + '>0,"海运上架",IF(AC' + str(i+3) + '<=0,"断货","")),"")' for i in range(len(df))]
df['实时库存'] = ['=IFERROR(AD' + str(i+3) + '-AB' + str(i+3) + ',"")' for i in range(len(df))]
df['Week-Sale'] = ['=SUM(AI' + str(i+3) + ':AO' + str(i+3) + ')' for i in range(len(df))]
df['可周转数'] = ['=IFERROR(AC' + str(i+3) + '/AP' + str(i+3) + ',"")' for i in range(len(df))]
df['干预判断'] = ['=IFERROR(IFS(AV' + str(i+3) + '<4,"需要干预",AND(AV' + str(i+3) + '>4,AX' + str(i+3) + '<>""),"恢复干预"),"")' for i in range(len(df))]
df['改价记录1'] = ['=AR' + str(i+3) + '&AY' + str(i+3) for i in range(len(df))]
df['本周改价记录1'] = ['=AS' + str(i+3) + '&AY' + str(i+3) for i in range(len(df))]

df

/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_42000/831898632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['累计涨价'] = ['=SUM(F' + str(i+3) + ':L' + str(i+3) + ')' for i in range(len(df))]
/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_42000/831898632.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['近7周平均销量'] = ['=IFERROR(ROUNDUP(AVERAGE(S' + str(i+3) + ':Y' + str(i+3) + '),1),"-")' for i in range(len(df))]


,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,Date_Right,累计涨价,近7周平均销量,改价,实时库存,Week-Sale,可周转数,干预判断,改价记录1,本周改价记录1
0,1,USACB00278,USACB00278,87.99,,,,,,,...,0.0,=SUM(F3:L3),"=IFERROR(ROUNDUP(AVERAGE(S3:Y3),1),""-"")","=IFERROR(IF(AC3-AE3>0,""海运上架"",IF(AC3<=0,""断货"",""""...","=IFERROR(AD3-AB3,"""")",=SUM(AI3:AO3),"=IFERROR(AC3/AP3,"""")","=IFERROR(IFS(AV3<4,""需要干预"",AND(AV3>4,AX3<>""""),""...",=AR3&AY3,=AS3&AY3
1,2,USACC68322,USACC68322-C,,60,15,,,,,...,1.0,=SUM(F4:L4),"=IFERROR(ROUNDUP(AVERAGE(S4:Y4),1),""-"")","=IFERROR(IF(AC4-AE4>0,""海运上架"",IF(AC4<=0,""断货"",""""...","=IFERROR(AD4-AB4,"""")",=SUM(AI4:AO4),"=IFERROR(AC4/AP4,"""")","=IFERROR(IFS(AV4<4,""需要干预"",AND(AV4>4,AX4<>""""),""...",=AR4&AY4,=AS4&AY4
2,3,USACC78321,USACC78321-C,,,,,,,,...,1.0,=SUM(F5:L5),"=IFERROR(ROUNDUP(AVERAGE(S5:Y5),1),""-"")","=IFERROR(IF(AC5-AE5>0,""海运上架"",IF(AC5<=0,""断货"",""""...","=IFERROR(AD5-AB5,"""")",=SUM(AI5:AO5),"=IFERROR(AC5/AP5,"""")","=IFERROR(IFS(AV5<4,""需要干预"",AND(AV5>4,AX5<>""""),""...",=AR5&AY5,=AS5&AY5
3,4,USACC1G010,USACC1G010-C,,10,,,,,,...,0.0,=SUM(F6:L6),"=IFERROR(ROUNDUP(AVERAGE(S6:Y6),1),""-"")","=IFERROR(IF(AC6-AE6>0,""海运上架"",IF(AC6<=0,""断货"",""""...","=IFERROR(AD6-AB6,"""")",=SUM(AI6:AO6),"=IFERROR(AC6/AP6,"""")","=IFERROR(IFS(AV6<4,""需要干预"",AND(AV6>4,AX6<>""""),""...",=AR6&AY6,=AS6&AY6
4,5,USACC98442,USACC98442-C,,,,,,,,...,0.0,=SUM(F7:L7),"=IFERROR(ROUNDUP(AVERAGE(S7:Y7),1),""-"")","=IFERROR(IF(AC7-AE7>0,""海运上架"",IF(AC7<=0,""断货"",""""...","=IFERROR(AD7-AB7,"""")",=SUM(AI7:AO7),"=IFERROR(AC7/AP7,"""")","=IFERROR(IFS(AV7<4,""需要干预"",AND(AV7>4,AX7<>""""),""...",=AR7&AY7,=AS7&AY7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,13747,USGS49520LR,USGS49520LR,,,,,,,,...,0.0,=SUM(F13749:L13749),"=IFERROR(ROUNDUP(AVERAGE(S13749:Y13749),1),""-"")","=IFERROR(IF(AC13749-AE13749>0,""海运上架"",IF(AC1374...","=IFERROR(AD13749-AB13749,"""")",=SUM(AI13749:AO13749),"=IFERROR(AC13749/AP13749,"""")","=IFERROR(IFS(AV13749<4,""需要干预"",AND(AV13749>4,AX...",=AR13749&AY13749,=AS13749&AY13749
13747,13748,USGS49650,USGS49650,30.69,,,,,,,...,0.0,=SUM(F13750:L13750),"=IFERROR(ROUNDUP(AVERAGE(S13750:Y13750),1),""-"")","=IFERROR(IF(AC13750-AE13750>0,""海运上架"",IF(AC1375...","=IFERROR(AD13750-AB13750,"""")",=SUM(AI13750:AO13750),"=IFERROR(AC13750/AP13750,"""")","=IFERROR(IFS(AV13750<4,""需要干预"",AND(AV13750>4,AX...",=AR13750&AY13750,=AS13750&AY13750
13748,13749,USGS64080,USGS64080,,,,,,,,...,0.0,=SUM(F13751:L13751),"=IFERROR(ROUNDUP(AVERAGE(S13751:Y13751),1),""-"")","=IFERROR(IF(AC13751-AE13751>0,""海运上架"",IF(AC1375...","=IFERROR(AD13751-AB13751,"""")",=SUM(AI13751:AO13751),"=IFERROR(AC13751/AP13751,"""")","=IFERROR(IFS(AV13751<4,""需要干预"",AND(AV13751>4,AX...",=AR13751&AY13751,=AS13751&AY13751
13749,13750,USGSM1048,USGSM1048,,,,,,,,...,0.0,=SUM(F13752:L13752),"=IFERROR(ROUNDUP(AVERAGE(S13752:Y13752),1),""-"")","=IFERROR(IF(AC13752-AE13752>0,""海运上架"",IF(AC1375...","=IFERROR(AD13752-AB13752,"""")",=SUM(AI13752:AO13752),"=IFERROR(AC13752/AP13752,"""")","=IFERROR(IFS(AV13752<4,""需要干预"",AND(AV13752>4,AX...",=AR13752&AY13752,=AS13752&AY13752


In [56]:
list_column = df.columns.tolist()
df = df[['序号', '层级', '在线SKU', 'SKU', '每周一大账号售价', '第一涨价', '第二涨价', '第三涨价', '第四涨价', '第五涨价', '第六涨价', '第七涨价', '累计涨价',
         '品类', '身份', 'MRP周销', '近7周平均销量', '未售出周数', 'W7', 'W6', 'W5', 'W4', 'W3', 'W2', 'W1',
         '在途总数', '改价', '无Record ID', '实时库存', '在库总数', 'Date_Left'] +
        list_column[12:21] +
        ['Date_Right', 'Week-Sale', '周销干预', '改价记录', '本周改价记录', '最新断货时间', '品类负责人', '可周转数', '干预判断', '干预记录',
         '提价', '改价记录1', '本周改价记录1', 'Note', '上次序号']]
list_column = df.columns.tolist()
list_column[list_column.index('在库总数')] = '当日库存'
df.columns = list_column

df.sort_values(by=['上次序号', 'SKU'], ascending=[True, True]).reset_index(drop=True)
df['序号'] = [i+1 for i in range(len(df))]

df

/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_42000/2265180725.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['序号'] = [i+1 for i in range(len(df))]


,序号,层级,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,...,最新断货时间,品类负责人,可周转数,干预判断,干预记录,提价,改价记录1,本周改价记录1,Note,上次序号
0,1,普通,USACB00278,USACB00278,87.99,,,,,,...,2023/05/18,Ada,"=IFERROR(AC3/AP3,"""")","=IFERROR(IFS(AV3<4,""需要干预"",AND(AV3>4,AX3<>""""),""...",,,=AR3&AY3,=AS3&AY3,,1
1,2,TOP10,USACC68322,USACC68322-C,,60,15,,,,...,2023/08/08,Gordon,"=IFERROR(AC4/AP4,"""")","=IFERROR(IFS(AV4<4,""需要干预"",AND(AV4>4,AX4<>""""),""...",,,=AR4&AY4,=AS4&AY4,,2
2,3,TOP10,USACC78321,USACC78321-C,,,,,,,...,2022/07/14,Gordon,"=IFERROR(AC5/AP5,"""")","=IFERROR(IFS(AV5<4,""需要干预"",AND(AV5>4,AX5<>""""),""...",,,=AR5&AY5,=AS5&AY5,,3
3,4,TOP20,USACC1G010,USACC1G010-C,,10,,,,,...,,Gordon,"=IFERROR(AC6/AP6,"""")","=IFERROR(IFS(AV6<4,""需要干预"",AND(AV6>4,AX6<>""""),""...",,,=AR6&AY6,=AS6&AY6,,4
4,5,普通,USACC98442,USACC98442-C,,,,,,,...,2022/07/12,Gordon,"=IFERROR(AC7/AP7,"""")","=IFERROR(IFS(AV7<4,""需要干预"",AND(AV7>4,AX7<>""""),""...",,,=AR7&AY7,=AS7&AY7,,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,13747,普通,USGS49520LR,USGS49520LR,,,,,,,...,,Ada,"=IFERROR(AC13749/AP13749,"""")","=IFERROR(IFS(AV13749<4,""需要干预"",AND(AV13749>4,AX...",,,=AR13749&AY13749,=AS13749&AY13749,,13747
13747,13748,普通,USGS49650,USGS49650,30.69,,,,,,...,,Ada,"=IFERROR(AC13750/AP13750,"""")","=IFERROR(IFS(AV13750<4,""需要干预"",AND(AV13750>4,AX...",,,=AR13750&AY13750,=AS13750&AY13750,,13748
13748,13749,普通,USGS64080,USGS64080,,,,,,,...,,Ada,"=IFERROR(AC13751/AP13751,"""")","=IFERROR(IFS(AV13751<4,""需要干预"",AND(AV13751>4,AX...",,,=AR13751&AY13751,=AS13751&AY13751,,13749
13749,13750,普通,USGSM1048,USGSM1048,,,,,,,...,,Ada,"=IFERROR(AC13752/AP13752,"""")","=IFERROR(IFS(AV13752<4,""需要干预"",AND(AV13752>4,AX...",,,=AR13752&AY13752,=AS13752&AY13752,,13750


In [57]:
for i in tqdm(range(len(df)), desc='Progress', ncols=77):
    try:
        df.loc[i, '每周一大账号售价'] = float(df.loc[i, '每周一大账号售价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第一涨价'] = float(df.loc[i, '第一涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第二涨价'] = float(df.loc[i, '第二涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第三涨价'] = float(df.loc[i, '第三涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第四涨价'] = float(df.loc[i, '第四涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第五涨价'] = float(df.loc[i, '第五涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第六涨价'] = float(df.loc[i, '第六涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第七涨价'] = float(df.loc[i, '第七涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'MRP周销'] = float(df.loc[i, 'MRP周销'].strip())
    except:
        pass
    
    try:
        df.loc[i, '未售出周数'] = int(df.loc[i, '未售出周数'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W7'] = int(df.loc[i, 'W7'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W6'] = int(df.loc[i, 'W6'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W5'] = int(df.loc[i, 'W5'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W4'] = int(df.loc[i, 'W4'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W3'] = int(df.loc[i, 'W3'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W2'] = int(df.loc[i, 'W2'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W1'] = int(df.loc[i, 'W1'].strip())
    except:
        pass
    
    try:
        df.loc[i, '在途总数'] = int(df.loc[i, '在途总数'].strip())
    except:
        pass
    
    try:
        df.loc[i, '无Record ID'] = int(df.loc[i, '无Record ID'].strip())
    except:
        pass
    
    try:
        df.loc[i, '当日库存'] = int(df.loc[i, '当日库存'].strip())
    except:
        pass
    
    try:
        df.iloc[i, 30] = int(df.iloc[i, 30].strip())
    except:
        pass
    
    try:
        df.iloc[i, 31] = int(df.iloc[i, 31].strip())
    except:
        pass
    
    try:
        df.iloc[i, 32] = int(df.iloc[i, 32].strip())
    except:
        pass
    
    try:
        df.iloc[i, 33] = int(df.iloc[i, 33].strip())
    except:
        pass
    
    try:
        df.iloc[i, 34] = int(df.iloc[i, 34].strip())
    except:
        pass
    
    try:
        df.iloc[i, 35] = int(df.iloc[i, 35].strip())
    except:
        pass
    
    try:
        df.iloc[i, 36] = int(df.iloc[i, 36].strip())
    except:
        pass
    
    try:
        df.iloc[i, 37] = int(df.iloc[i, 37].strip())
    except:
        pass
    
    try:
        df.iloc[i, 38] = int(df.iloc[i, 38].strip())
    except:
        pass
    
    try:
        df.iloc[i, 39] = int(df.iloc[i, 39].strip())
    except:
        pass
    
    try:
        df.iloc[i, 40] = int(df.iloc[i, 40].strip())
    except:
        pass
    
df.to_excel('./output/am-colorless.xlsx', index=False, startrow=1)

df

Progress: 100%|███████████████████████| 13751/13751 [00:18<00:00, 734.95it/s]


,序号,层级,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,...,最新断货时间,品类负责人,可周转数,干预判断,干预记录,提价,改价记录1,本周改价记录1,Note,上次序号
0,1,普通,USACB00278,USACB00278,87.99,,,,,,...,2023/05/18,Ada,"=IFERROR(AC3/AP3,"""")","=IFERROR(IFS(AV3<4,""需要干预"",AND(AV3>4,AX3<>""""),""...",,,=AR3&AY3,=AS3&AY3,,1
1,2,TOP10,USACC68322,USACC68322-C,,60.0,15.0,,,,...,2023/08/08,Gordon,"=IFERROR(AC4/AP4,"""")","=IFERROR(IFS(AV4<4,""需要干预"",AND(AV4>4,AX4<>""""),""...",,,=AR4&AY4,=AS4&AY4,,2
2,3,TOP10,USACC78321,USACC78321-C,,,,,,,...,2022/07/14,Gordon,"=IFERROR(AC5/AP5,"""")","=IFERROR(IFS(AV5<4,""需要干预"",AND(AV5>4,AX5<>""""),""...",,,=AR5&AY5,=AS5&AY5,,3
3,4,TOP20,USACC1G010,USACC1G010-C,,10.0,,,,,...,,Gordon,"=IFERROR(AC6/AP6,"""")","=IFERROR(IFS(AV6<4,""需要干预"",AND(AV6>4,AX6<>""""),""...",,,=AR6&AY6,=AS6&AY6,,4
4,5,普通,USACC98442,USACC98442-C,,,,,,,...,2022/07/12,Gordon,"=IFERROR(AC7/AP7,"""")","=IFERROR(IFS(AV7<4,""需要干预"",AND(AV7>4,AX7<>""""),""...",,,=AR7&AY7,=AS7&AY7,,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,13747,普通,USGS49520LR,USGS49520LR,,,,,,,...,,Ada,"=IFERROR(AC13749/AP13749,"""")","=IFERROR(IFS(AV13749<4,""需要干预"",AND(AV13749>4,AX...",,,=AR13749&AY13749,=AS13749&AY13749,,13747
13747,13748,普通,USGS49650,USGS49650,30.69,,,,,,...,,Ada,"=IFERROR(AC13750/AP13750,"""")","=IFERROR(IFS(AV13750<4,""需要干预"",AND(AV13750>4,AX...",,,=AR13750&AY13750,=AS13750&AY13750,,13748
13748,13749,普通,USGS64080,USGS64080,,,,,,,...,,Ada,"=IFERROR(AC13751/AP13751,"""")","=IFERROR(IFS(AV13751<4,""需要干预"",AND(AV13751>4,AX...",,,=AR13751&AY13751,=AS13751&AY13751,,13749
13749,13750,普通,USGSM1048,USGSM1048,,,,,,,...,,Ada,"=IFERROR(AC13752/AP13752,"""")","=IFERROR(IFS(AV13752<4,""需要干预"",AND(AV13752>4,AX...",,,=AR13752&AY13752,=AS13752&AY13752,,13750


# 6. 格式更改

In [58]:
from openpyxl import load_workbook

In [59]:
wb_last = load_workbook(last)
ws_last = wb_last.worksheets[0]

wb_this = load_workbook('./output/am-colorless.xlsx')
ws_this = wb_this.worksheets[0]

In [60]:
coef = 1.15
ws_this.column_dimensions['A'].width = coef * 6
ws_this.column_dimensions['B'].width = coef * 9
ws_this.column_dimensions['C'].width = coef * 15
ws_this.column_dimensions['D'].width = coef * 15
ws_this.column_dimensions['E'].width = coef * 9
ws_this.column_dimensions['F'].width = coef * 8
ws_this.column_dimensions['G'].width = coef * 8
ws_this.column_dimensions['H'].width = coef * 8
ws_this.column_dimensions['I'].width = coef * 8
ws_this.column_dimensions['J'].width = coef * 8
ws_this.column_dimensions['K'].width = coef * 8
ws_this.column_dimensions['L'].width = coef * 8
ws_this.column_dimensions['M'].width = coef * 9
ws_this.column_dimensions['N'].width = coef * 9
ws_this.column_dimensions['O'].width = coef * 9
ws_this.column_dimensions['P'].width = coef * 7
ws_this.column_dimensions['Q'].width = coef * 7
ws_this.column_dimensions['R'].width = coef * 5
ws_this.column_dimensions['S'].width = coef * 5
ws_this.column_dimensions['T'].width = coef * 5
ws_this.column_dimensions['U'].width = coef * 5
ws_this.column_dimensions['V'].width = coef * 5
ws_this.column_dimensions['W'].width = coef * 5
ws_this.column_dimensions['X'].width = coef * 5
ws_this.column_dimensions['Y'].width = coef * 5
ws_this.column_dimensions['Z'].width = coef * 7
ws_this.column_dimensions['AA'].width = coef * 7
ws_this.column_dimensions['AB'].width = coef * 10
ws_this.column_dimensions['AC'].width = coef * 10
ws_this.column_dimensions['AD'].width = coef * 10
ws_this.column_dimensions['AE'].width = coef * 10
ws_this.column_dimensions['AF'].width = coef * 10
ws_this.column_dimensions['AG'].width = coef * 10
ws_this.column_dimensions['AH'].width = coef * 10
ws_this.column_dimensions['AI'].width = coef * 10
ws_this.column_dimensions['AJ'].width = coef * 10
ws_this.column_dimensions['AK'].width = coef * 10
ws_this.column_dimensions['AL'].width = coef * 10
ws_this.column_dimensions['AM'].width = coef * 10
ws_this.column_dimensions['AN'].width = coef * 10
ws_this.column_dimensions['AO'].width = coef * 10
ws_this.column_dimensions['AP'].width = coef * 8
ws_this.column_dimensions['AQ'].width = coef * 10
ws_this.column_dimensions['AR'].width = coef * 10
ws_this.column_dimensions['AS'].width = coef * 10
ws_this.column_dimensions['AT'].width = coef * 10
ws_this.column_dimensions['AU'].width = coef * 8
ws_this.column_dimensions['AV'].width = coef * 8
ws_this.column_dimensions['AW'].width = coef * 8
ws_this.column_dimensions['AX'].width = coef * 8
ws_this.column_dimensions['AY'].width = coef * 8
ws_this.column_dimensions['AZ'].width = coef * 8
ws_this.column_dimensions['BA'].width = coef * 8
ws_this.column_dimensions['BB'].width = coef * 10
ws_this.column_dimensions['BC'].width = coef * 6

ws_this.row_dimensions[1].height = 25
ws_this.row_dimensions[2].height = 34
for i in range(3, ws_this.max_row+1):
    ws_this.row_dimensions[i].height = 16.5

In [61]:
from openpyxl.styles import Font, PatternFill, Alignment

In [62]:
ws_this['A1'].value = str(datetime.now().month) + '/' + str(datetime.now().day)
ws_this['A1'].font = Font(b=True)
ws_this['A1'].number_format = 'yyyy/m/d;@'

ws_this['C1'].value = '每周二删除本周改价记录栏'
ws_this['C1'].font = Font(bold=True, color='FFFF0000')
ws_this['C1'].fill = PatternFill(patternType='solid', fgColor='FFFFFF00')

ws_this['D1'].value = '基本信息'
ws_this['D1'].font = Font(b=True)
ws_this.merge_cells('D1:R1')

ws_this['S1'].value = '近7周 周销情况'
ws_this.merge_cells('S1:Y1')

ws_this['AA1'].value = '任务栏'
ws_this['AA1'].font = Font(b=True)
ws_this['AA1'].fill = PatternFill(patternType='solid', fgColor='FF92D050')

ws_this['AB1'].value = '=SUM(AB3:AB' + str(ws_this.max_row) + ')'
ws_this['AB1'].font = Font(b=True, color='FFFF0000')

ws_this['AC1'].value = '=SUM(AC3:AC' + str(ws_this.max_row) + ')'
ws_this['AC1'].font = Font(b=True, color='FFFF0000')

ws_this['AD1'].value = '=SUM(AD3:AD' + str(ws_this.max_row) + ')'
ws_this['AD1'].font = Font(b=True, color='FFFF0000')

ws_this['AE1'].value = '=SUM(AE3:AE' + str(ws_this.max_row) + ')'
ws_this['AE1'].font = Font(b=True, color='FFFF0000')

ws_this['AF1'].value = '=SUM(AF3:AF' + str(ws_this.max_row) + ')'
ws_this['AF1'].font = Font(b=True, color='FFFF0000')

ws_this['AG1'].value = '=SUM(AG3:AG' + str(ws_this.max_row) + ')'
ws_this['AG1'].font = Font(b=True, color='FFFF0000')

ws_this['AH1'].value = '=SUM(AH3:AH' + str(ws_this.max_row) + ')'
ws_this['AH1'].font = Font(b=True, color='FFFF0000')

ws_this['AR1'].value = '格式案例：10.19-改0; 9.23-T5; 9.30-H; 10.5-J3; 10.18-T6(日期，最新在后面）'

ws_this['AV1'].value = '实时库存≤5，周销不为0，或者周销为0，无record不为0的，在线都已改为实时库存，且已关闭自动补货'
ws_this['AV1'].fill = PatternFill(patternType='solid', fgColor='FFFFFF00')

In [63]:
for list_cell in ws_this['A2:R2']:
    for cell in list_cell:
        cell.fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')
        
for list_cell in ws_this['Z2:AA2']:
    for cell in list_cell:
        cell.fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')
        
ws_this['AB2'].fill = PatternFill(patternType='solid', fgColor='FFFFFF00')

ws_this['AC2'].fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')

ws_this['AD2'].fill = PatternFill(patternType='solid', fgColor='FF00B0F0')

for list_cell in ws_this['AE2:AH2']:
    for cell in list_cell:
        cell.number_format = 'yyyy/m/d;@'
        cell.fill = PatternFill(patternType='solid', fgColor='FF00B0F0')

for list_cell in ws_this['AI2:AO2']:
    for cell in list_cell:
        cell.number_format = 'yyyy/m/d;@'
        cell.fill = PatternFill(patternType='solid', fgColor='FFFABF8E')

for list_cell in ws_this['AP2:AQ2']:
    for cell in list_cell:
        cell.font = Font(color='FFFFFFFF')
        cell.fill = PatternFill(patternType='solid', fgColor='FF963634')

for list_cell in ws_this['AR2:AU2']:
    for cell in list_cell:
        cell.fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')

for list_cell in ws_this['AV2:AX2']:
    for cell in list_cell:
        cell.font = Font(color='FFFFFFFF')
        cell.fill = PatternFill(patternType='solid', fgColor='FFE26B0A')

for list_cell in ws_this['AY2:BA2']:
    for cell in list_cell:
        cell.fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')

In [64]:
for i in tqdm(range(3, ws_this.max_row+1), desc='Progress', ncols=77):
    ws_this['E' + str(i)].number_format = '\\$#,##0.00;\\-\\$#,##0.00'
    
    no = ws_this['BC' + str(i)].value
    if no == None:
        continue
    
    no = int(no) + 2
    if ws_last['AG' + str(no)].font.color != None and type(ws_last['AG' + str(no)].font.color.rgb) == str:
        ws_this['AH' + str(i)].font = Font(color=ws_last['AG' + str(no)].font.color.rgb)
    if ws_last['AF' + str(no)].font.color != None and type(ws_last['AF' + str(no)].font.color.rgb) == str:
        ws_this['AG' + str(i)].font = Font(color=ws_last['AF' + str(no)].font.color.rgb)
    if ws_last['AE' + str(no)].font.color != None and type(ws_last['AE' + str(no)].font.color.rgb) == str:
        ws_this['AF' + str(i)].font = Font(color=ws_last['AE' + str(no)].font.color.rgb)
    if ws_last['AD' + str(no)].font.color != None and type(ws_last['AD' + str(no)].font.color.rgb) == str:
        ws_this['AE' + str(i)].font = Font(color=ws_last['AD' + str(no)].font.color.rgb)
        ws_this['AD' + str(i)].font = Font(color=ws_last['AD' + str(no)].font.color.rgb)

Progress: 100%|█████████████████████| 13751/13751 [00:00<00:00, 14144.69it/s]


In [65]:
wb_this.save('./断货改价-2U-上午 ' + str(datetime.now().month) + '.' + str(datetime.now().day) + '.xlsx')
wb_this.close()
wb_last.close()

print('Done ~')

Done ~
